In [1]:
import pandas as pd
import numpy as np
import xgboost 
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier

/Users/kayibal/virtualenvs/ml_project/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_hdf("/srv/smsguru/dataset/embedded_full.hdf5")

In [3]:
df = df.groupby("question_id").mean().reset_index()
df = df.iloc[np.random.permutation(np.arange(len(df)))]

In [4]:
train = df.iloc[:int(len(df)*0.6)]
tune = df.iloc[int(len(df)*0.6):int(len(df)*0.6)+int(len(df)*0.1)]
test = df.iloc[int(len(df)*0.6)+int(len(df)*0.1):]

In [5]:
Y_ALL = train[["question_id", "parent_id"]].drop_duplicates().set_index("question_id")

In [6]:
feature_cols = ["question_id"] + ["dim_{}".format(i) for i in range(300)]
X = train[feature_cols].set_index("question_id")

In [8]:
Y_ALL.parent_id.value_counts()

(14,)

In [ ]:
targets = Y_ALL.parent_id.value_counts().index.tolist()

In [16]:
def evaluate_tree():
    for i in targets:
        Y = (Y_ALL.parent_id == 9).astype(int)
        mdl = DecisionTreeClassifier()
        mdl.fit(np.array(X, dtype=np.int32), np.array(Y, dtype=np.int32))
        Y_TEST = test[["question_id", "parent_id"]].drop_duplicates().set_index("question_id")
        Y_true = (Y_TEST.parent_id == 16).astype(int)
        X_test = test[feature_cols].set_index("question_id")
        pred = mdl.predict(np.array(X_test, dtype=np.int32))
        print(i, roc_auc_score(Y_true, pred))

In [17]:
def evaluate_sgd():
    for i in targets:
        Y = (Y_ALL.parent_id == 9).astype(int)
        mdl = SGDClassifier()
        mdl.fit(np.array(X, dtype=np.int32), np.array(Y, dtype=np.int32))
        Y_TEST = test[["question_id", "parent_id"]].drop_duplicates().set_index("question_id")
        Y_true = (Y_TEST.parent_id == 16).astype(int)
        X_test = test[feature_cols].set_index("question_id")
        pred = mdl.predict(np.array(X_test, dtype=np.int32))
        print(i, roc_auc_score(Y_true, pred))

In [18]:
evaluate_sgd()

16.0 0.5
11.0 0.5
8.0 0.5
15.0 0.5
13.0 0.5
14.0 0.5
7.0 0.5
17.0 0.5
74.0 0.5
12.0 0.5
10.0 0.5
18.0 0.5
19.0 0.5
9.0 0.5


In [19]:
Y = (Y_ALL.parent_id == 9).astype(int)
mdl = DecisionTreeClassifier()
mdl.fit(np.array(X, dtype=np.int32), np.array(Y, dtype=np.int32))
Y_TEST = test[["question_id", "parent_id"]].drop_duplicates().set_index("question_id")
Y_true = (Y_TEST.parent_id == 16).astype(int)
X_test = test[feature_cols].set_index("question_id")
pred = mdl.predict(np.array(X_test, dtype=np.int32))
roc_auc_score(Y_true, pred)

0.5

In [21]:
mdl = {}
for i in targets:
    Y = (Y_ALL.parent_id == i).astype(int)

    d = xgboost.DMatrix(X, label=Y)

    mdl[i] = xgboost.train({'max_depth':10, 'eta':0.1, 'silent':1, 'objective':'binary:logistic' }, d, 100)

    Y_TEST = test[["question_id", "parent_id"]].drop_duplicates().set_index("question_id")

    Y_true = (Y_TEST.parent_id == i).astype(int)

    X_test = test[feature_cols].set_index("question_id")

    pred = mdl[i].predict(xgboost.DMatrix(X_test))

    print(i,roc_auc_score(Y_true, pred))

16.0 0.783187013181
11.0 0.977756298963
8.0 0.900721144783
15.0 0.863864133357
13.0 0.836002032518
14.0 0.849958905361
7.0 0.877309147228
17.0 0.888266629359
74.0 0.825702806099
12.0 0.874259934484
10.0 0.796752805893
18.0 0.880897932735
19.0 0.890293497291
9.0 0.854018121734


In [20]:
mapping = list(Y_ALL.parent_id.unique())

In [21]:
def map_labels(x):
    return mapping.index(x) + 1

In [24]:
Y = Y_ALL.parent_id.apply(map_labels)

d = xgboost.DMatrix(X, label=Y)

mdl = xgboost.train({'max_depth':10, 'eta':0.1, 'silent':1, 'objective':'multi:softmax', 'num_class':15 }, d, 100)

Y_TEST = test[["question_id", "parent_id"]].drop_duplicates().set_index("question_id")

NameError: name 'i' is not defined

In [26]:
Y_true = Y_TEST.parent_id

X_test = test[feature_cols].set_index("question_id")

pred = mdl.predict(xgboost.DMatrix(X_test))

In [27]:
accuracy_score(pred, Y_TEST)

NameError: name 'accuracy_score' is not defined

In [23]:
def multicls_pred(X):
    pred = {}
    for target, model in mdl.items():
        pred[target] = model.predict(xgboost.DMatrix(X))
    return pred

In [24]:
X_test = test[feature_cols].set_index("question_id")
pred = multicls_pred(X_test)
pred = pd.DataFrame(pred).idxmax(axis=1)

In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(pred, Y_TEST)

0.086966551326412914